In [1]:
#####---チェーンウォークMDP問題---###

import numpy as np

#状態遷移行列
Pas=np.array([[0.9, 0.1,0,0],[0.1,0.9,0,0],[0.9,0.1,0,0],[0,0.1,0.9,0],
            [0,0.9,0.1,0],[0,0,0.1,0.9],[0,0,0.9,0.1],[0,0,0.1,0.9]])
print(Pas)

[[ 0.9  0.1  0.   0. ]
 [ 0.1  0.9  0.   0. ]
 [ 0.9  0.1  0.   0. ]
 [ 0.   0.1  0.9  0. ]
 [ 0.   0.9  0.1  0. ]
 [ 0.   0.   0.1  0.9]
 [ 0.   0.   0.9  0.1]
 [ 0.   0.   0.1  0.9]]


In [2]:
#政策の行列表現

pL=0.1
pi=np.zeros(32).reshape(4,8)
for i in range(4):
    pi[i,2*i]=pL
    pi[i,2*i+1]=1-pL

print(pi)
    

[[ 0.1  0.9  0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.1  0.9  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.1  0.9  0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.1  0.9]]


In [3]:
#状態推移確率行列
Mss=np.dot(pi,Pas)
print(Mss)
print()
#(状態・行動)対の推移確率行列
Maa = np.dot(Pas,pi)
print(Maa)

[[ 0.18  0.82  0.    0.  ]
 [ 0.09  0.1   0.81  0.  ]
 [ 0.    0.09  0.1   0.81]
 [ 0.    0.    0.18  0.82]]

[[ 0.09  0.81  0.01  0.09  0.    0.    0.    0.  ]
 [ 0.01  0.09  0.09  0.81  0.    0.    0.    0.  ]
 [ 0.09  0.81  0.01  0.09  0.    0.    0.    0.  ]
 [ 0.    0.    0.01  0.09  0.09  0.81  0.    0.  ]
 [ 0.    0.    0.09  0.81  0.01  0.09  0.    0.  ]
 [ 0.    0.    0.    0.    0.01  0.09  0.09  0.81]
 [ 0.    0.    0.    0.    0.09  0.81  0.01  0.09]
 [ 0.    0.    0.    0.    0.01  0.09  0.09  0.81]]


In [4]:
#報酬ベクトル (状態がS4の時のみ報酬)
#推移確率行列にRを書けた数値が報酬の期待値
Ra=np.array([0,0,0,0,0,0,1,1])
Rs=np.array([0,0,0,1])

#割引率
gamma=0.9

In [5]:
###解析的に解く方法###

#tステップ後の利得はγ^(t-1) * R * Psa^t = R/γ*(γPsa)^t
#無限等比級数の和の形。行列ではΣ(γPsa)^t=((I-γPsa)^(-1)-I)と計算できる.
from numpy.linalg import inv
I8=np.identity(8)
I4=np.identity(4)

A=I4-gamma*Mss
Vlt=np.dot((inv(A)-I4),Rs)/gamma
print(Vlt)

B=I8-gamma*Maa
Qlt=np.dot((inv(B)-I8),Ra)/gamma
print()
print(Qlt.reshape(4,2))

[ 6.08650719  6.91123716  7.95091733  8.04598705]

[[ 5.55208217  6.14588775]
 [ 5.55208217  7.06225438]
 [ 6.31368466  8.13283207]
 [ 7.26438187  8.13283207]]


In [6]:
###方策評価###

V=np.zeros(4).reshape(4,1) #価値関数をゼロで初期化
theta=0.000000001
delta=theta
i=0

while(delta>=theta and i<100000):
    Vnext=Rs.reshape(4,1)+gamma*V #4次元の列ベクトル
    Vupdate=np.dot(Mss,Vnext) #価値関数の更新：状態遷移行列を前からかける
    delta=max(np.absolute(Vupdate-V))
    V=Vupdate
    i=i+1

print(i)
print(V)

196
[[ 6.08650718]
 [ 6.91123715]
 [ 7.95091732]
 [ 8.04598704]]


In [7]:
###方策改善###
Q=np.dot(Pas, Rs.reshape(4,1)+gamma*V).reshape(4,2)
am=Q.argmax(1)
print(am)

#方策PiをGreedyに更新
pi=np.zeros(32).reshape(4,8)
for i in range(4):
    if am[i]==0: #Qが最大になるのがLの時,決定的にLを選択
        pi[i,2*i]=1
        pi[i,2*i+1]=0
    else: #Qが最大になるのがRの時,決定的にRを選択
        pi[i,2*i]=0
        pi[i,2*i+1]=1

print(pi)


[1 1 1 1]
[[ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]]
